# Анализ сегментации городов (1991)

Данные Union Bank о 48 городах; сегменты по экономическим условиям и краткая интерпретация.

**План исследования:**
- первичная проверка и очистка данных;
- обработка пропусков и выбросов;
- разведочный анализ и визуализации;
- сегментация KMeans и иерархическая кластеризация;
- интерпретация кластеров и (опционально) факторный анализ.

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import FactorAnalysis
from scipy import stats
import scipy.cluster.hierarchy as sch

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.titlesize"] = 12
plt.rcParams["axes.labelsize"] = 10


## 1. Загрузка и предварительный осмотр данных

In [ ]:
data_path = Path("Econom_Cities_data.csv")
df_raw = pd.read_csv(
    data_path,
    sep=";",
    decimal=",",
    na_values=-9999
)

print(f"Размер исходного датасета: {df_raw.shape[0]} наблюдений, {df_raw.shape[1]} показателей")
display(df_raw.head())

In [ ]:

summary = df_raw.describe().T
missing = df_raw.isna().sum()

display(summary)
display(missing.to_frame(name="missing"))


## 2. Очистка данных и проверка выбросов

In [ ]:
numeric_cols = ["Work", "Price", "Salary"]

missing_cities = df_raw.loc[df_raw[numeric_cols].isna().any(axis=1), "City"].tolist()
df = df_raw.dropna(subset=numeric_cols).reset_index(drop=True)

if missing_cities:
    print(f"Удаляем {len(missing_cities)} города(ов) с пропусками: {', '.join(missing_cities)}")
else:
    print("Пропусков в целевых показателях не обнаружено.")

z_scores = np.abs(stats.zscore(df[numeric_cols]))
df["is_outlier"] = (z_scores > 3).any(axis=1)

outlier_cities = df.loc[df["is_outlier"], "City"].tolist()
if outlier_cities:
    print(f"Найдены выбросы (|z|>3): {', '.join(outlier_cities)}")
else:
    print("Выбросы по критерию |z|>3 не обнаружены.")

df["outlier_label"] = np.where(df["is_outlier"], "выброс", "норма")
display(df.head())

In [ ]:
df_model = df.loc[~df["is_outlier"]].copy()
df_model = df_model.drop(columns=["is_outlier"]).reset_index(drop=True)

print(f"Датасет для кластеризации: {df_model.shape[0]} городов")
display(df_model.head())

### Разведочный анализ

In [ ]:
axes = df_model[numeric_cols].hist(figsize=(10, 3), bins=8)
plt.suptitle("Распределение экономических показателей", y=1.02)
plt.tight_layout()

In [ ]:
pairplot_data = df.copy()
sns.pairplot(
    data=pairplot_data,
    vars=numeric_cols,
    hue="outlier_label",
    diag_kind="hist",
    plot_kws={"alpha": 0.75, "s": 50}
)
plt.suptitle("Парные отношения показателей", y=1.02)

In [ ]:

df_model = df_model.drop(columns=["outlier_label"], errors="ignore")


## 3. Стандартизация и выбор числа кластеров

In [ ]:
features = numeric_cols.copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_model[features])

print("Пример стандартизированных значений:")
display(pd.DataFrame(X_scaled[:5], columns=[f"z_{col}" for col in features]))

In [ ]:
k_values = range(2, 7)
metrics = []

for k in k_values:
    model = KMeans(n_clusters=k, n_init=20, random_state=42)
    labels = model.fit_predict(X_scaled)
    inertia = model.inertia_
    silhouette = silhouette_score(X_scaled, labels)
    metrics.append({"k": k, "inertia": inertia, "silhouette": silhouette})

metrics_df = pd.DataFrame(metrics)
display(metrics_df)

best_k_silhouette = metrics_df.loc[metrics_df["silhouette"].idxmax(), "k"]
print(f"Максимальное значение силуэта достигается при k = {best_k_silhouette:.0f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(metrics_df["k"], metrics_df["inertia"], marker="o")
axes[0].set_title("Метод локтя (инерция)")
axes[0].set_xlabel("k")
axes[0].set_ylabel("Inertia")

axes[1].plot(metrics_df["k"], metrics_df["silhouette"], marker="o", color="tab:green")
axes[1].set_title("Средний силуэт")
axes[1].set_xlabel("k")
axes[1].set_ylabel("Silhouette")

for ax in axes:
    ax.set_xticks(list(k_values))

plt.tight_layout()

## 4. KMeans-кластеризация и интерпретация

In [ ]:
optimal_k = 3
print(
    "Используем k = {0} для финальной модели (учитывая баланс между силуэтом, методом локтя и бизнес-логикой).".format(optimal_k)
)

kmeans = KMeans(n_clusters=optimal_k, n_init=20, random_state=42)
df_model["kmeans_cluster"] = kmeans.fit_predict(X_scaled)

cluster_centers = pd.DataFrame(
    scaler.inverse_transform(kmeans.cluster_centers_),
    columns=features
).round(1)
cluster_centers.index.name = "kmeans_cluster"
display(cluster_centers)

In [ ]:

cluster_counts = df_model["kmeans_cluster"].value_counts().sort_index()
cluster_profile = df_model.groupby("kmeans_cluster")[features].mean().round(1)
cluster_profile["cities_count"] = cluster_counts
cluster_profile["examples"] = (
    df_model.groupby("kmeans_cluster")["City"].apply(lambda s: ", ".join(s.sort_values().head(3)))
)
cluster_profile = cluster_profile.sort_values("Price")
display(cluster_profile)


In [ ]:
centers_plot = cluster_centers.copy()
centers_plot = centers_plot.reset_index().rename(columns={"kmeans_cluster": "cluster"})

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.scatterplot(
    data=df_model,
    x="Salary",
    y="Price",
    hue="kmeans_cluster",
    palette="Set2",
    s=80,
    ax=axes[0]
)
sns.scatterplot(
    data=centers_plot,
    x="Salary",
    y="Price",
    hue="cluster",
    palette="Set2",
    s=160,
    marker="X",
    ax=axes[0],
    legend=False
)
axes[0].set_title("Salary vs Price")

sns.scatterplot(
    data=df_model,
    x="Work",
    y="Salary",
    hue="kmeans_cluster",
    palette="Set2",
    s=80,
    ax=axes[1]
)
sns.scatterplot(
    data=centers_plot,
    x="Work",
    y="Salary",
    hue="cluster",
    palette="Set2",
    s=160,
    marker="X",
    ax=axes[1],
    legend=False
)
axes[1].set_title("Work vs Salary")

for ax in axes:
    ax.legend(title="Кластер")

plt.tight_layout()

**Интерпретация KMeans-сегментов:**
- `kmeans_cluster = 1`: города с высокими ценами и зарплатами (Zurich, Tokyo, Geneva, Oslo). Жители работают относительно меньше часов, но получают сошно больше среднего.
- `kmeans_cluster = 0`: города со средним уровнем цен и умеренными зарплатами; много европейских и латиноамериканских локаций (Athens, Lisbon, Mexico City).
- `kmeans_cluster = 2`: города с длинным рабочим днём и невысокими зарплатами, в основном Азия и Латинская Америка (Bogota, Manila, Hong Kong).

## 5. Иерархическая кластеризация (Ward)

In [ ]:
linkage_matrix = sch.linkage(X_scaled, method="ward")
color_threshold = linkage_matrix[-(optimal_k - 1), 2]

plt.figure(figsize=(8, 10))
sch.dendrogram(
    linkage_matrix,
    labels=df_model["City"].values,
    orientation="right",
    color_threshold=color_threshold
)
plt.title("Дендограмма (Ward linkage)")
plt.xlabel("Евклидово расстояние")
plt.tight_layout()

In [ ]:

agg = AgglomerativeClustering(n_clusters=optimal_k, linkage="ward")
df_model["hier_cluster"] = agg.fit_predict(X_scaled)

cross_tab = pd.crosstab(df_model["kmeans_cluster"], df_model["hier_cluster"])
display(cross_tab)

hier_profile = df_model.groupby("hier_cluster")[features].mean().round(1)
display(hier_profile)


**Наблюдения по иерархической модели:** иерархическая кластеризация даёт схожее разбиение: один кластер с высокими ценами/зарплатами, два — с более доступной стоимостью жизни; Ward-алгоритм группирует часть городов с высокой нагрузкой по часам, что полезно для альтернативной интерпретации.

## 6. Дополнительно: факторный анализ

In [ ]:
fa = FactorAnalysis(n_components=2, random_state=42)
factor_scores = fa.fit_transform(X_scaled)

loadings = pd.DataFrame(
    fa.components_,
    columns=features,
    index=["Factor 1", "Factor 2"]
).round(2)
display(loadings)

factor_df = pd.DataFrame(factor_scores, columns=["Factor 1", "Factor 2"])
factor_df["City"] = df_model["City"].values
factor_df["kmeans_cluster"] = df_model["kmeans_cluster"].values

display(factor_df.head())

plt.figure(figsize=(6, 5))
sns.scatterplot(
    data=factor_df,
    x="Factor 1",
    y="Factor 2",
    hue="kmeans_cluster",
    palette="Set2",
    s=80
)
plt.title("Города в факторном пространстве")
plt.tight_layout()